<a href="https://colab.research.google.com/github/arnold-798/Projects_v1/blob/main/peer_bank_call_1q21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Peer Bank Analysis

In [ ]:
# If first time running program, run this cell, if not, skip cell 
!pip install PyPDF4
!pip install urllib3
!pip install PDFMiner

     |████████████████████████████████| 71kB 3.2MB/s 
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-cp37-none-any.whl size=61249 sha256=974d146fd972fbfac33de87d33597d7d96850fe63d848646a1e599396b31d8d7
  Stored in directory: /root/.cache/pip/wheels/eb/4f/15/c64d533cb496fd874f56045fe30e8cc0ac59f99ecdd718040d
Successfully built PyPDF4
     |████████████████████████████████| 4.2MB 5.7MB/s 
     |████████████████████████████████| 1.9MB 43.5MB/s 
  Created wheel for PDFMiner: filename=pdfminer-20191125-cp37-none-any.whl size=6140081 sha256=439df407304206d0451db87e0f86b778fbf63e19f6d4e155557cd9899c0331ad
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built PDFMiner


In [ ]:
# Import Packages

import os
import re
import sys
import requests
import numpy as np
import pandas as pd
import PyPDF4 as pdf
from bs4 import BeautifulSoup as bs
import urllib3

In [ ]:
# Import packages to read pdf files and extract text

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

Replace the previous quarter's pdf links


> You can find the links to the peer bank's in the peer analysis folder on the shared drive


In [ ]:
!pwd # Identify the current working directory

/content


In [ ]:
# Create a working directory for the quarterly earnings releases

peer_folder_path = "/content/peer_earnings/2020Q4" # Change the ending quarter to the current quarter

try:
    os.makedirs(peer_folder_path)
except OSError:
    print ("Creation of the directory %s failed" % peer_folder_path)
else:
    print ("Successfully created the directory %s" % peer_folder_path)

os.chdir(peer_folder_path) # If using Colab it should say './content/'; if using Jupiter Notebook or Spyder, it may point to your documents folder or desktop

Successfully created the directory /content/peer_earnings/2020Q4


In [ ]:
!pwd # Check that the created directory is your current working directory

/content/peer_earnings/2020Q4


In [ ]:
# Upload Excel File or Update Links Below

# If using colab, generally try to import into the colab folders and set the link to /content/.... 
# If using Jupiter notebook or Spyder, set link to where the file is located 
    # FYI: You may run into some issues setting the file path to the shared due to access issues (takes longer to pull data from; or not allowed to pull directly from shared drive)
    # PSA: Generally easier and faster to copy excel file to your desktop and set link to there

bank_links_path = "~/Users/chrisarnold/Desktop/" 

bank_links_file_name = "peer_bank_earnings_calls.xlsx"

bank_links = os.path.join(bank_links_path, bank_links_file_name).replace('\\','/')

#bank_links_df = pd.read_csv(bank_links) # Unblock these when / if you choose to import the excel file

In [ ]:
# 2020 Q4 Bank Links

# Do not run this cell if you are importing an excel file with links 

cfg = 'https://investor.citizensbank.com/~/media/Files/C/CitizensBank-IR/reports-and-presentations/financials/2020/4q-2020-financial-supplement.pdf'
usb = 'https://ir.usbank.com/static-files/380e7cc3-2a12-4ade-bb4e-5e8111e81a58'
rf = 'https://ir.regions.com/~/media/Files/R/Regions-IR/documents/RF-20201231-Exhibit-992%20FINAL-website.pdf'
fitb = 'https://s23.q4cdn.com/252949160/files/doc_financials/2020/q4/Q4-2020-Earnings-Release.pdf'
tfc = 'https://ir.truist.com/download/TFC+4Q20+Earnings+Presentation.pdf'
hban = 'http://huntington-ir.com/ne/news/hban420.pdf'
key = 'https://s23.q4cdn.com/646737342/files/doc_financials/2020/q4/4Q20-Earnings-Release-vF.pdf'
pnc = 'https://thepncfinancialservicesgroupinc.gcs-web.com/static-files/d88f22c5-0af3-4bb4-8345-0aa8af720e68'
cma = 'https://investor.comerica.com/download/Comerica+4Q20+Press+Release.pdf'
zions = 'https://zionsbancorporation.com/files/doc_news/2021/01/National-Association-Reports-Fourth-Quarter-and-Full-Year-2020-Financial-Results.pdf'
wfc = 'https://www.wellsfargo.com/assets/pdf/about/investor-relations/earnings/fourth-quarter-2020-earnings-supplement.pdf'
bac = 'https://d1io3yog0oux5.cloudfront.net/_6fcfa8c3def3f59d07243789a91041dc/bankofamerica/db/806/9028/earnings_release/The+Press+Release.pdf'
c = 'https://www.citigroup.com/citi/investor/data/qer420s.pdf?ieNocache=279'
jpm = 'https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/quarterly-earnings/2020/4th-quarter/64d9321a-23db-401f-89de-a259ac985044.pdf'
ms = 'https://www.morganstanley.com/about-us-ir/finsup4q2020/finsup4q2020.pdf'
gs = 'https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2020-q4-results.pdf'

bank_ticker = pd.Series(["CFG", "USB", "RF", "FITB", "TFC", "HBAN", "KEY", "PNC", "CMA", "ZIONS", "WFC", "BAC", "C", "JPM", "MS", "GS"], name = 'Bank_Ticker')

earnings_links_2020q4 = pd.Series([cfg, usb, rf, fitb, tfc, hban, key, pnc, cma, zions, wfc, bac, c, jpm, ms, gs], name='Current_Quarter_Links')

earnings_links_2020q4 = pd.concat([bank_ticker, earnings_links_2020q4], axis = 1)

start_earnings_2020q4 = pd.DataFrame(earnings_links_2020q4)

start_earnings_2020q4

,Bank_Ticker,Current_Quarter_Links
0,CFG,https://investor.citizensbank.com/~/media/File...
1,USB,https://ir.usbank.com/static-files/380e7cc3-2a...
2,RF,https://ir.regions.com/~/media/Files/R/Regions...
3,FITB,https://s23.q4cdn.com/252949160/files/doc_fina...
4,TFC,https://ir.truist.com/download/TFC+4Q20+Earnin...
5,HBAN,http://huntington-ir.com/ne/news/hban420.pdf
6,KEY,https://s23.q4cdn.com/646737342/files/doc_fina...
7,PNC,https://thepncfinancialservicesgroupinc.gcs-we...
8,CMA,https://investor.comerica.com/download/Comeric...
9,ZIONS,https://zionsbancorporation.com/files/doc_news...


Build Out Functions to Download the PDFs
> The function downloads PDF files from their links and then writes them out to the working directory folder

In [ ]:
# Import Packages to Download PDF Files from Internet

import urllib.request

In [ ]:
# Download Earnings Release PDFs

def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

In [ ]:
# Test out the function with CFG 

cfg_2020_q4_link = 'https://investor.citizensbank.com/~/media/Files/C/CitizensBank-IR/reports-and-presentations/financials/2020/4q-2020-financial-supplement.pdf'

download_file(cfg_2020_q4_link, "CFG_2020_q4")

Build Out Functions to Extract Text from Earnings Release PDFs
> First Tested with CFG Earnings Release

In [ ]:
# CFG Test

cfg_2020_q4 = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'

In [ ]:
# https://dzone.com/articles/exporting-data-from-pdfs-with-python - Website where base code was taken

# miner_text_generator.py

import io

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage


# miner_text_generator.py

def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

# Extract all text from each page and print
    # This is for testing purposes while building out the model
    # Eventually will need to extend the function to decifer and organize text within the pdf

def extract_text(pdf_path):
    for page in extract_text_by_page(pdf_path):
      
      
      #page_df = pd.concat(page_v1)
      return page

if __name__ == '__main__':
    print(extract_text(cfg_2020_q4))

Financial SupplementFourth Quarter and Full Year 2020 1


In [ ]:
if __name__=='__main__':
  test = (extract_text_by_page(cfg_2020_q4))

type(test)

generator

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFTextExtractionNotAllowed

path = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'

# Create a pdf document parser with file objects
Praser = PDFParser(open(path, 'rb'))
# Create a PDF document
doc = PDFDocument()
# Connection Analyzer and Document Objects
parser.set_document(doc)
Doc.set_parser(parser)

# Provide an initialization password
# Create an empty string if there is no password
doc.initialize()

# Detect if the document provides a txt conversion, ignore it if not provided
if not doc.is_extractable:
    raise PDFTextExtractionNotAllowed
else:
    # Create PDf Explorer to manage shared resources
    Rsrcmgr = PDFResourceManager()
    # Create a PDF device object
    Laparams = LAParams()
    Device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    # Create a PDF interpreter object
    Interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Loop through the list, processing the content of one page at a time
    for page in doc.get_pages():
        Interpreter.process_page(page)
        # Accept the LTPage object for this page
        Layout = device.get_result()
        # Here layout is an LTPage object that stores the various objects parsed by this page
        # Includes LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal, etc. 
        for x in layout:
            if isinstance(x, LTTextBox):
                print(x.get_text().strip())

TypeError: ignored

In [ ]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 5.6MB 4.8MB/s 
     |████████████████████████████████| 143kB 51.3MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.27-cp37-none-any.whl size=32071 sha256=30e830789847bc5007ab52b57f11924e40708907d28c62b4b8d92f7646c47ec6
  Stored in directory: /root/.cache/pip/wheels/b4/74/fc/f7b3a1a0732209027fb48a5f4392fc40d79970b11c2ba49e71
Successfully built pdfplumber


In [ ]:

import pdfplumber

Path = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'
Pdf = pdfplumber.open(path)

for page in Pdf.pages:
    # Get all the text information for the current page, including the text in the table
     print(page.extract_text()) 
     
     for table in page.extract_tables():
        # print(table)
        for row in table:
            print(row)
        print('-- -------- Split line----------')

Pdf.close()


Financial Supplement
Fourth Quarter and Full Year 2020 
1
Table of Contents Page
Consolidated Financial Highlights 3
Consolidated Statements of Operations (unaudited) 5
Consolidated Balance Sheets (unaudited) 6
Loans and Deposits 7
Average Balance Sheets 8
Average Annualized Yields and Rates 9
Segment Financial Highlights 10
Credit-Related Information:
Nonaccrual loans and leases 14
Loans and Leases 90 Days or More Past Due and Accruing 15
Charge-offs, Recoveries, and Related Ratios 16
Summary of Changes in the Components of the Allowance for Credit Losses 18
Capital and Ratios 19
Non-GAAP Financial Measures and Reconciliations  20
The information in this Financial Supplement is preliminary and based on company data available at the time of 
the earnings presentation.  It speaks only as of the particular date or dates included in the accompanying 
pages.  The Company does not undertake an obligation to, and disclaims any duty to, update any of the 
information provided.  Any forward-lo

In [ ]:
!pip uninstall camelot
!pip uninstall camelot-py[cv]
!pip uninstall camelot-py[all]

Uninstalling Camelot-12.6.29:
  Would remove:
    /usr/local/bin/camelot_admin
    /usr/local/bin/camelot_example
    /usr/local/bin/camelot_mini_example
    /usr/local/lib/python3.7/dist-packages/Camelot-12.6.29.dist-info/*
    /usr/local/lib/python3.7/dist-packages/camelot/*
    /usr/local/lib/python3.7/dist-packages/camelot_example/*
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/action_button.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/admin_classes.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/budget.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/camelot-new-project.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/camelot_qt_linguist.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/collection_proxy.png
    /usr/local/lib/python3.7/dist-packages/doc/sphinx/source/_static/color.png
    /usr/local/lib/python3.7/dist-packages/doc/sphin

In [ ]:
!pip install tabula
!pip install camelot-py
!pip install camelot-py[cv]
!pip install camelot-py[all]
!pip install pdfminer.six

  Using cached https://files.pythonhosted.org/packages/ed/78/123007d6aff9811bd087ff5dd68e06a9df62b2fdbf7685d74c9d6f247921/camelot_py-0.8.2-py3-none-any.whl


In [ ]:
!pip install ghostscript
!brew install ghostscript


/bin/bash: brew: command not found


In [ ]:
https://www.thepythoncode.com/article/extract-pdf-tables-in-python-camelot

<module 'camelot' from '/usr/local/lib/python3.7/dist-packages/camelot/__init__.py'>

In [ ]:
import camelot.io as camelot
from ghostscript install 

file = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'

tables = camelot.read_pdf(file)

tables

RuntimeError: ignored

In [ ]:
import tabula


Path = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'

Df = tabula.read_pdf(path, encoding='gbk', pages='all')
for indexs in df. Index:
    print(df.loc[indexs].values)

tabula.convert_into(path, os.path.splitext(path)[0]+'. Csv', pages='all')

AttributeError: ignored

In [ ]:
!pip install 

ERROR: Could not find a version that satisfies the requirement PDFResourceManager (from versions: none)
ERROR: No matching distribution found for PDFResourceManager


In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice

In [ ]:
# Open a PDF file.
fp = open('/content/peer_earnings/2020Q4/CFG_2020_q4.pdf', 'rb')
# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)
# Create a PDF document object that stores the document structure.
# Supply the password for initialization.
document = PDFDocument(parser, password='')
# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()
# Create a PDF device object.
device = PDFDevice(rsrcmgr)
# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)

In [ ]:
!pip install pdfminer 

Processing /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326/pdfminer-20191125-cp37-none-any.whl


In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTTextBoxHorizontal
from pdfminer.pdfpage import PDFPage


document = open('/content/peer_earnings/2020Q4/CFG_2020_q4.pdf', 'rb')
#Create resource manager
rsrcmgr = PDFResourceManager()
# Set parameters for analysis.
laparams = LAParams()
# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
for page in PDFPage.get_pages(document):
    interpreter.process_page(page)
    # receive the LTPage object for the page.
    layout = device.get_result()
    for element in layout:
        if isinstance(element, LTTextBoxHorizontal):
            print(element.get_text())

Streaming output truncated to the last 5000 lines.
 

 

 
 
 

 
 
 
 

 
 
 
 

 
 
 
 
 

 
 
 

$2,542 

— 
— 

— 

2,542 

145 
87 
232 

5 
37 
42 
190 

84 
7 
91 
$2,443 

$194 
— 
33 
$227 
$2,670 

$1,419 
1,251 
$2,670 

 

 
 

 

 

 
 
 

 
 
 
 

 
 
 
 

 
 
 
 
 

 
 
 

$2,448 

— 
— 

— 

2,448 

171 
86 
257 

1 
37 
38 
219 

224 
89 
313 
$2,542 

$79 
— 
115 
$194 
$2,736 

$1,441 
1,295 
$2,736 

 

 
 

 

 

 
 
 

 
 
 
 

 
 
 
 

 
 
 
 
 

 
 
 

$2,171 

— 
— 

— 

2,171 

74 
106 
180 

3 
30 
33 
147 

554 
(130) 
424 
$2,448 

$39 
— 
40 
$79 
$2,527 

$1,304 
1,223 
$2,527 

 

 
 

 

 

 
 
 

 
 
 
 

 
 
 
 

 
 
 
 
 

 
 
 

$1,252 

(176) 
629 

453 

1,705 

47 
127 
174 

3 
34 
37 
137 

298 
305 
603 
$2,171 

$44 
(2) 
(3) 
$39 
$2,210 

$790 
1,420 
$2,210 

 

 
 

 

 

 
 
 

 
 
 
 

 
 
 
 

 
 
 
 
 

 
 
 

$1,263 

— 
— 

— 

1,263 

34 
128 
162 

7 
33 
40 
122 

(11) 
122 
111 
$1,252 

$45 
— 
(1) 
$44 
$1,296 

$718 
578 
$1,

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

# Open a PDF document.
fp = open('/content/peer_earnings/2020Q4/CFG_2020_q4.pdf', 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser, password='')

# Get the outlines of the document.
outlines = document.get_outlines()
for (level,title,dest,a,se) in outlines:
    print (level, title)



1 Financial Supplement
2 Cover
2 Table of Contents
2 Consolidated Financial Highlights
2 Consolidated Statements of Operations
2 Consolidated Balance Sheets
2 Loans and Deposits
2 Average Balance Sheets 
2 Average Annualized Yields
2 Segment Financial Highlights
2 Nonaccrual loans and leases
2 Loans and Leases 90 Days or More Past Due and Accruing
2 Charge-offs, Recoveries and Related Ratios 
2 Summary of Changes in the Components of the Allowance for Credit Losses
2 Capital and Ratios
2 Key Performance Metrics, Non-GAAP Financial Measures
2 Key Performance Metrics, Non-GAAP Financial Measures - Segments


In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Open a PDF file.
fp = open('/content/peer_earnings/2020Q4/CFG_2020_q4.pdf', 'rb')
# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)
# Create a PDF document object that stores the document structure.
# Supply the password for initialization.
document = PDFDocument(parser, password='')
# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
  raise PDFTextExtractionNotAllowed
# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()
# Create a PDF device object.
device = PDFDevice(rsrcmgr)
# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in PDFPage.create_pages(document):
  interpreter.process_page(page)


In [ ]:
cfg_test = pd.DataFrame(extract_text(cfg_2020_q4))
cfg_test['Financial']

Financial SupplementFourth Quarter and Full Year 2020 1

Table of ContentsPageConsolidated Financial Highlights3Consolidated Statements of Operations (unaudited)5Consolidated Balance Sheets (unaudited)6Loans and Deposits7Average Balance Sheets8Average Annualized Yields and Rates9Segment Financial Highlights10Credit-Related Information:Nonaccrual loans and leases14Loans and Leases 90 Days or More Past Due and Accruing15Charge-offs, Recoveries, and Related Ratios16Summary of Changes in the Components of the Allowance for Credit Losses18Capital and Ratios19Non-GAAP Financial Measures and Reconciliations 20The information in this Financial Supplement is preliminary and based on company data available at the time of the earnings presentation.  It speaks only as of the particular date or dates included in the accompanying pages.  The Company does not undertake an obligation to, and disclaims any duty to, update any of the information provided.  Any forward-looking statements in this Financi

KeyError: ignored

In [ ]:
# json_exporter.py

import json
import os

def export_as_json(pdf_path, json_path):
    filename = os.path.splitext(os.path.basename(pdf_path))[0]
    data = {'Filename': filename}
    data['Pages'] = []

    counter = 1
    for page in extract_text_by_page(pdf_path):
        text = page[0:1000000]
        page = {'Page_{}'.format(counter): text}
        data['Pages'].append(page)
        counter += 1

    with open(json_path, 'w') as fh:
        json.dump(data, fh)

if __name__ == '__main__':
    pdf_path = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'
    json_path = '/content/peer_earnings/2020Q4/cfg_2020_q4.json'
    export_as_json(pdf_path, json_path)

In [ ]:
# csv_exporter.py

import csv
import os

def export_as_csv(pdf_path, csv_path):
    filename = os.path.splitext(os.path.basename(pdf_path))[0]

    counter = 1
    with open(csv_path, 'w') as csv_file:
        writer = csv.writer(csv_file)
        for page in extract_text_by_page(pdf_path):
            text = page[0:1000000]
            words = text.split()
            writer.writerow(words)
            print()


if __name__ == '__main__':
    pdf_path = '/content/peer_earnings/2020Q4/CFG_2020_q4.pdf'
    csv_path = '/content/peer_earnings/2020Q4/cfg_2020_q4.csv'
    export_as_csv(pdf_path, csv_path)

In [ ]:
cfg = '/content/peer_earnings/2020Q4/cfg_2020_q4.csv'

cfg_xl = pd.read_csv(cfg, sep = ',')

ParserError: ignored

In [ ]:
# Explore the exported CSV

def import_csv(file_path):
  pdf_csv = pd.read_csv(file_path)
  rdf_df = pd.DataFrame(pdf_csv)
  return rdf_df

In [ ]:
def bank_release_data(xlsx_file_path, export_path):
  pd.read_csv()

Alternative Method V3

In [ ]:
!pip uninstall pdfminer
!pip uninstall slate3k
# (Optional - In my case I also had to uninstall pdfminer3, so:) 3. pip uninstall pdfminer3

!pip install slate3k

In [ ]:
# slate_text_extraction.py

!pip install https://github.com/timClicks/slate/archive/master.zip
!pip install utils

import slate

  Using cached https://github.com/timClicks/slate/archive/master.zip
  Created wheel for slate: filename=slate-0.5.2-cp37-none-any.whl size=18183 sha256=397a9ff417dafc4afda7dda90a29d9d4d783c4f4d901c6ca0874544024e35897
  Stored in directory: /tmp/pip-ephem-wheel-cache-fp0kly6k/wheels/74/e8/2c/ea67445a8f160ee922447f33ac6f768ee0244b4962db8d5fc3
Successfully built slate


In [ ]:
# Extracting Data with Slate (More intuitive, ultimately uses PDFMiner as a base)

import slate3k

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as fh:
        document = slate3k.PDF(fh)

    for page in document:
        print(page)

In [ ]:
# Test out the pdf reader functi

extract_text_from_pdf(cfg_2020_q4)


TypeError: ignored

Alternative Method V1


In [ ]:
from PyPDF4 import PdfFileReader

def extract_information(pdf_path):

  pdf = download_pdf(pdf_path, file_name)
  pdf = PdfFileReader(f)
  information = pdf.getDocumentInfo()
  number_of_pages = pdf.getNumPages()

  txt = f"""
  Information about {pdf_path}: 

  Author: {information.author}
  Creator: {information.creator}
  Producer: {information.producer}
  Subject: {information.subject}
  Title: {information.title}
  Number of pages: {number_of_pages}
  """

  print(txt)
  return information

IndentationError: ignored

Final Function
> Put the created functions together to iterate across




In [ ]:
#earnings_release_20q4 = pd.concat(map(lambda x: download_file(x['Current_Quarter_Links'],x['Bank_Ticker']), start_earnings_2020q4))

Export RAW PDF Results to Individual CSVs

In [ ]:
import csv
import os

from miner_text_generator import extract_text_by_page


def export_as_csv(pdf_path, csv_path):
    filename = os.path.splitext(os.path.basename(pdf_path))[0]

    counter = 1
    with open(csv_path, 'w') as csv_file:
        writer = csv.writer(csv_file)
        for page in extract_text_by_page(pdf_path):
            text = page[0:100]
            words = text.split()
            writer.writerow(words)


if __name__ == '__main__':
    pdf_path = 'w9.pdf'
    csv_path = 'w9.csv'
    export_as_csv(pdf_path, csv_path)